# 让终端的输出有颜色

In [9]:
pip install -U colorful-prints

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [1]:
import colorful_prints as cp

In [3]:
cp.warning("hello world")

hello world

In [4]:
cp.info("hello world")

hello world

In [5]:
cp.warning("!!!!")

!!!!

In [6]:
cp.danger("error")

error

In [7]:
cp.success("pass")

pass

In [8]:
cp.white_print("green")

green

# 下面时可以调用的所有函数

 `colorful_print`
 
 `yellow_print`
 
 `red_print`
 
 `green_print`
 
 `blue_print`
 
 `magenta_print`
 
 `cyan_print`
 
 `white_print`
 
 `black_print`
 
 `bright_red_print`
 
 `bright_green_print`
 
 `bright_blue_print`
 
 `bright_yellow_print`
 
 `bright_magenta_print`
 
 `bright_cyan_print`
 
 `bright_white_print`
 
 `dim_red_print`
 
 `dim_green_print`
 
 `dim_blue_print`
 
 `dim_yellow_print`
 
 `dim_magenta_print`
 
 `dim_cyan_print`
 
 `dim_white_print`
 
 `danger`
 
 `success`
 
 `info`
 
 `warning`
 

In [9]:
cp.__dir__()

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__path__',
 '__file__',
 '__cached__',
 '__builtins__',
 'colorful_print',
 'yellow_print',
 'red_print',
 'green_print',
 'blue_print',
 'magenta_print',
 'cyan_print',
 'white_print',
 'black_print',
 'bright_red_print',
 'bright_green_print',
 'bright_blue_print',
 'bright_yellow_print',
 'bright_magenta_print',
 'bright_cyan_print',
 'bright_white_print',
 'dim_red_print',
 'dim_green_print',
 'dim_blue_print',
 'dim_yellow_print',
 'dim_magenta_print',
 'dim_cyan_print',
 'dim_white_print',
 'danger',
 'success',
 'info',
 'warning']

# 使用对话机器人进行测试

In [ ]:
"""
Author: hxy
Date: 2025/02/23
Description: 使用langgraph构建llm
"""

# %%
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
import colorful_prints as cp

model = "deepseek-v3-free"
llm = ChatOpenAI(model=model)
# llm.invoke("你好，你是什么模型？")
# %%
_debug = False


def concat(original: list, new: list) -> list:
    if _debug:
        # print(f"left: {original}\nright:{new}")
        cp.info(f"left: {original}\nright:{new}")
    return original + new


class ChatState(TypedDict):
    messages: Annotated[list, concat]  # messages的状态和reduce function


def chat(state: ChatState):
    answer = llm.invoke(state["messages"])
    return {"messages": [answer]}


workflow = StateGraph(ChatState)
workflow.add_node(chat)
workflow.set_entry_point("chat")
workflow.add_edge("chat", "__end__")
graph = workflow.compile(checkpointer=MemorySaver())


def session(id):
    return {"configurable": {"thread_id": str(id)}}


while True:
    user_input = input(">>> ")

    r = []
    for message_chunk, metadata in graph.stream(
        {"messages": [HumanMessage(user_input)]},
        config=session(1),
        stream_mode="messages",
    ):
        if message_chunk.content:
            r.append(message_chunk.content)
            # print(message_chunk.content, end="", flush=True)
            cp.success(message_chunk.content, end="")
    print()
    r = " ".join(r)
    # r = graph.stream({"messages": [HumanMessage(user_input)]}, config=session(1))
    # print("AI: " + r)

    if _debug:
        # print("-------------")
        cp.white_print("-------------")
        for h in graph.get_state_history(session(1)):
            # print(h, end="\n\n")
            cp.info(h, end="\n\n")
        cp.white_print("-------------")
        # print("-------------")